`conda env create -n suite2p_env -f optinist/conda/suite2p_env.yaml`

`!pip install imageio pynwb`

In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from optinist.wrappers.suite2p_wrapper import file_convert, registration, roi
from optinist.api.dataclass.dataclass import ImageData

In [2]:
sample_data = ImageData(['../sample_data/mouse2p_2_donotouse.tiff'], "")

In [3]:
file_convert_params = {
    'nplanes': 1,
    'nchannels': 1,
    'force_sktiff': False,
    'batch_size': 500,
    'do_registration': 1
}

In [4]:
ret = file_convert.suite2p_file_convert(sample_data, file_convert_params)
ret

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally
start suite2_file_convert
<class 'list'>
../sample_data/mouse2p_2_donotouse.tiff
['../sample_data']
['mouse2p_2_donotouse.tiff']
tif
** Found 1 tifs - converting to binary **
NOTE: ScanImageTiffReader not working for this tiff type, using tifffile


{'meanImg': <optinist.api.dataclass.image.ImageData at 0x7fcc73963250>,
 'ops': <optinist.api.dataclass.suite2p.Suite2pData at 0x7fcc820cbcd0>}

In [5]:
registration_params = {
    'frames_include': -1,
    'keep_movie_raw': False,
    'do_bidiphase': False,

    'smooth_sigma': 1.15,
    'smooth_sigma_time': 0,
    'bidiphase': 0,
    'maxregshift': 0.1,
    'maxregshiftNR': 5,
    'nonrigid': True,
    'block_size': [128, 128],
    'snr_thresh': 1.2,
    'functional_chan' : 1,
    'align_by_chan' : 1,
    'reg_tif': False,
    'th_badframes': 1.0,
    'diameter': 0,

    # 1P setting
    '1Preg': False,
    'spatial_hp_reg': 42,
    'pre_smooth': 0,
    'spatial_taper': 40,
    'bidi_corrected': False,
}

In [6]:
ret = registration.suite2p_registration(ret['ops'], registration_params)

start suite2_registration
{'suite2p_version': '0.10.3', 'look_one_level_down': False, 'fast_disk': '/tmp/optinist/output/suite2p/plane0', 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'bruker_bidirectional': False, 'h5py': [], 'h5py_key': 'data', 'nwb_file': '', 'nwb_driver': '', 'nwb_series': '', 'save_path0': '/tmp/optinist/output', 'save_folder': 'suite2p', 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 10.0, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': False, 'ignore_flyback': [], 'preclassify': 0.0, 'save_mat': False, 'save_NWB': False, 'combined': True, 'aspect': 1.0, 'do_bidiphase': False, 'bidiphase': 0, 'bidi_corrected': False, 'do_registration': 1, 'two_step_registration': False, 'keep_movie_raw': False, 'nimg_init': 300, 'batch_size': 500, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': False, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0, 'smooth_sigma': 1.15, '

/Users/keita/opt/anaconda3/envs/suite2p_env_38/lib/python3.8/site-packages/suite2p/registration/utils.py:234: RuntimeWarning: invalid value encountered in apply_dotnorm
  return ifft2(apply_dotnorm(fft2(mov), img)) #.astype(np.complex64)


added enhanced mean image


In [7]:
suite2p_roi_params = {
    # main settings
    'tau':  1.0,              # this is the main parameter for deconvolution
    'fs': 10.0,             # sampling rate (PER PLANE e.g. for 12 plane recordings it will be around 2.5)

    # classification parameters
    'soma_crop': True,        # crop dendrites for cell classification stats like compactness

    # cell detection settings
    'high_pass': 100,         # running mean subtraction with window of size 'high_pass(use low values for 1P)
    'sparse_mode': True,      # whether or not to run sparse_mode
    'max_overlap': 0.75,      # cells with more overlap than this get removed during triage before refinement
    'nbinned': 5000,          # max number of binned frames for cell detection
    'spatial_scale': 0,       # 0: multi-scale; 1: 6 pixels 2: 12 pixels 3: 24 pixels 4: 48 pixels
    'threshold_scaling': 1.0, # adjust the automatically determined threshold by this scalar multiplier
    'max_iterations': 20,     # maximum number of iterations to do cell detection

    # 1P settings
    'spatial_hp_detect': 25,  # window for spatial high-pass filtering for neuropil subtraction before detection

    # output settings
    'preclassify': 0.,       # apply classifier before signal extraction with probability 0.3

    # ROI extraction parameters
    'allow_overlap': False,      # pixels that are overlapping are thrown out (False) or added to both ROIs (True)
    'inner_neuropil_radius': 2,  # number of pixels to keep between ROI and neuropil donut
    'min_neuropil_pixels': 350,  # minimum number of pixels in the neuropil

    # deconvolution settings
    'neucoeff': .7,          # neuropil coefficient
}

In [8]:
ret = roi.suite2p_roi(ret['ops'], suite2p_roi_params)

start suite2p_roi
NOTE: Applying builtin classifier at /Users/keita/opt/anaconda3/envs/suite2p_env_38/lib/python3.8/site-packages/suite2p/classifiers/classifier.npy
Binning movie in chunks of length 10
Binned movie [100,122,126] in 0.02 sec.
NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 5.00 
0 ROIs, score=62.26


/Users/keita/opt/anaconda3/envs/suite2p_env_38/lib/python3.8/site-packages/suite2p/detection/sparsedetect.py:256: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  im, _ = mode(imap[ipk][isort[:50]])
/Users/keita/opt/anaconda3/envs/suite2p_env_38/lib/python3.8/site-packages/suite2p/detection/sparsedetect.py:270: UserWarning: Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.
  warn("Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.")


Detected 187 ROIs, 0.49 sec
After removing overlaps, 177 ROIs remain
roi extraction
Masks created, 0.07 sec.
Extracted fluorescence from 177 ROIs in 1000 frames, 2.49 sec.
roi classification
['compact', 'npix_norm', 'skew']


/Users/keita/Code/optinist_test/optinist/wrappers/suite2p_wrapper/roi.py:101: RuntimeWarning: All-NaN slice encountered
  'all_roi': RoiData(np.nanmax(im, axis=0), file_name='all_roi'),
/Users/keita/Code/optinist_test/optinist/wrappers/suite2p_wrapper/roi.py:102: RuntimeWarning: All-NaN slice encountered
  'non_cell_roi': RoiData(np.nanmax(im[~iscell], axis=0), file_name='noncell_roi'),
/Users/keita/Code/optinist_test/optinist/wrappers/suite2p_wrapper/roi.py:103: RuntimeWarning: All-NaN slice encountered
  'cell_roi': RoiData(np.nanmax(im[iscell], axis=0), file_name='cell_roi'),


In [19]:
for roi in ret['nwbfile']['ROI']['roi_list']:
    print(roi.keys())

dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['pixel_mask'])
dict_keys(['